In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import matplotlib.pyplot as plt
from google.colab import drive, files
from datetime import datetime
import seaborn as sns

In [2]:
drive.mount('/content/drive')
df = pd.read_excel("/content/drive/MyDrive/Bootcamp Llamadas Cobranza/Transcripciones_Variables.xlsx")

Mounted at /content/drive


In [3]:
df = df[['archivo','duracion_seg','transcripcion','CONN_ID','2.2 Alternativas de acuerdo a la necesidad']]
df = df.dropna(subset=["2.2 Alternativas de acuerdo a la necesidad"])

In [4]:
!pip install -U "transformers>=4.40" "datasets>=2.18" "peft>=0.11" "accelerate>=0.30" -q
import transformers; print("Transformers:", transformers.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 82.9 MB/s eta 0:00:00
Transformers: 4.56.1


In [5]:
import os
os.environ["WANDB_DISABLED"] = "true"   # apaga W&B
os.environ["WANDB_MODE"] = "disabled"   # redundante pero ayuda

In [6]:
# =========================================================
# Helpers + run_lora_experiments (compatibles y listos)
# =========================================================
import warnings, os, inspect, re
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")

# ---- HuggingFace / LoRA ----
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer
)
from peft import LoraConfig, get_peft_model, TaskType

# ---- Métricas / splits ----
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_recall_curve, auc,
    f1_score, accuracy_score, precision_score, recall_score,
    confusion_matrix
)

# ---- Progreso (opcional) ----
try:
    from tqdm.auto import tqdm
    _HAS_TQDM = True
except Exception:
    _HAS_TQDM = False

RSEED = 42
np.random.seed(RSEED)

# ---------------------------------------------------------
# Compatibilidad con versiones antiguas de transformers
# ---------------------------------------------------------
def build_training_args(
    output_dir="./_lora_tmp",
    num_train_epochs=5,
    lr=2e-5,
    batch_size=8,
    weight_decay=0.01,
    seed=42,
    logging_steps=50
):
    return TrainingArguments(
        output_dir=output_dir,
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=weight_decay,
        logging_steps=logging_steps,
        seed=seed,
        report_to=[]  # 🔒 evita que intente loguearse a wandb o tb
    )

# ---------------------------------------------------------
# Utils de métricas / oversampling simple / threshold
# ---------------------------------------------------------
def tune_threshold_on_val(y_val, scores_val):
    """Devuelve el umbral que maximiza F1 de la clase positiva en validación."""
    prec, rec, thr = precision_recall_curve(y_val, scores_val)
    f1 = (2 * prec * rec) / (prec + rec + 1e-12)
    best_idx = int(np.nanargmax(f1[:-1])) if len(thr) > 0 else 0
    return float(thr[best_idx]) if len(thr) > 0 else 0.5

def _sigmoid(x):
    return 1 / (1 + np.exp(-x))

def _oversample_dataframe(df_in, label_col, ratio=1.0, random_state=RSEED):
    """
    Oversample por replicación de la clase 1 hasta alcanzar:
      n_pos_final ≈ ratio * n_neg
    ratio=1.0 => 50/50 aprox.
    """
    if ratio is None:
        return df_in
    df = df_in.copy()
    y = df[label_col].astype(int).values
    n_pos, n_neg = (y == 1).sum(), (y == 0).sum()
    if n_pos == 0 or n_neg == 0:
        return df
    target_pos = int(np.round(ratio * n_neg))
    if target_pos <= n_pos:
        return df
    extra = df[df[label_col] == 1].sample(
        n=target_pos - n_pos, replace=True, random_state=random_state
    )
    return pd.concat([df, extra], ignore_index=True)

def make_metrics_row(y_true, y_pred, y_score, dataset, model, sampler, vectorizer,
                     threshold, n_features, notes=""):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    acc       = accuracy_score(y_true, y_pred)
    f1_micro  = f1_score(y_true, y_pred, average="micro", zero_division=0)
    f1_macro  = f1_score(y_true, y_pred, average="macro", zero_division=0)
    f1_weight = f1_score(y_true, y_pred, average="weighted", zero_division=0)
    prec_1    = precision_score(y_true, y_pred, pos_label=1, zero_division=0)
    rec_1     = recall_score(y_true, y_pred,    pos_label=1, zero_division=0)
    f1_1      = f1_score(y_true, y_pred,        pos_label=1, zero_division=0)
    prec_0    = precision_score(y_true, y_pred, pos_label=0, zero_division=0)
    rec_0     = recall_score(y_true, y_pred,    pos_label=0, zero_division=0)
    f1_0      = f1_score(y_true, y_pred,        pos_label=0, zero_division=0)
    prec, rec, _ = precision_recall_curve(y_true, y_score)
    pr_auc = auc(rec, prec)
    return pd.DataFrame([{
        "dataset": dataset,
        "model": model,
        "sampler": sampler,
        "vectorizer": vectorizer,
        "threshold": float(threshold),
        "n_features": int(n_features),
        "n_samples": int(len(y_true)),
        "prevalence_1": float((np.array(y_true) == 1).mean()),
        "accuracy": acc,
        "f1_micro": f1_micro,
        "f1_macro": f1_macro,
        "f1_weighted": f1_weight,
        "pr_auc": pr_auc,
        "precision_1": prec_1, "recall_1": rec_1, "f1_1": f1_1,
        "precision_0": prec_0, "recall_0": rec_0, "f1_0": f1_0,
        "tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp),
        "notes": notes
    }])

# ---------------------------------------------------------
# Experimentos LoRA con ids de modelos correctos
# ---------------------------------------------------------
def run_lora_experiments(
    df,
    text_col="transcripcion",
    target_col="2.2 Alternativas de acuerdo a la necesidad",
    base_models=None,
    lora_grid=None,
    test_size=0.2,
    val_size=0.25,
    max_length=256,
    num_epochs=5,
    lr=2e-5,
    batch_size=8,
    weight_decay=0.01,
    oversample_ratio=None,
    verbose=True
):
    """
    Fine-tuning con LoRA, optimiza threshold en validación y devuelve un DataFrame
    con la estructura de make_metrics_row (dataset='test').
    """
    # Modelos por defecto (IDs válidos y públicos)
    if base_models is None:
        base_models = [
            "PlanTL-GOB-ES/roberta-base-bne",
            "dccuchile/bert-base-spanish-wwm-cased",
            "xlm-roberta-base"
        ]
    # Parrilla LoRA por defecto
    if lora_grid is None:
        lora_grid = [
            {"r": 16, "alpha": 32, "dropout": 0.1},
            {"r": 8,  "alpha": 16, "dropout": 0.0}
        ]

    # Limpieza df
    assert text_col in df.columns and target_col in df.columns, "Columnas no encontradas."
    dfx = df.dropna(subset=[text_col, target_col]).copy()
    dfx[target_col] = dfx[target_col].astype(int)

    # Split train / test
    X = dfx[text_col].astype(str).values
    y = dfx[target_col].astype(int).values
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=test_size, stratify=y, random_state=RSEED)

    df_train = pd.DataFrame({text_col: X_tr, target_col: y_tr})
    resultados = []

    combos = [(bm, g["r"], g["alpha"], g["dropout"]) for bm in base_models for g in lora_grid]
    iterator = tqdm(combos) if (_HAS_TQDM and verbose) else combos

    for base_name, r, alpha, dropout in iterator:
        tag = f"{os.path.basename(base_name)}|r={r}|a={alpha}|d={dropout}"
        if verbose and not _HAS_TQDM:
            print("▶️", tag)
        try:
            # Oversampling (train)
            df_tr_os = _oversample_dataframe(df_train, target_col, ratio=oversample_ratio, random_state=RSEED)
            X_tr_t, X_val, y_tr_t, y_val = train_test_split(
                df_tr_os[text_col].astype(str).values,
                df_tr_os[target_col].astype(int).values,
                test_size=val_size,
                stratify=df_tr_os[target_col].values,
                random_state=RSEED
            )

            # Datasets HF
            df_tr_hf = pd.DataFrame({text_col: X_tr_t, "label": y_tr_t})
            df_val_hf= pd.DataFrame({text_col: X_val,  "label": y_val})
            df_te_hf = pd.DataFrame({text_col: X_te,   "label": y_te})

            ds_train = Dataset.from_pandas(df_tr_hf, preserve_index=False)
            ds_val   = Dataset.from_pandas(df_val_hf, preserve_index=False)
            ds_test  = Dataset.from_pandas(df_te_hf,  preserve_index=False)

            # Tokenizer (no pasar token=None)
            tok = AutoTokenizer.from_pretrained(base_name, use_fast=True)
            def _tok(batch):
                return tok(batch[text_col], truncation=True, padding="max_length", max_length=max_length)
            ds_train = ds_train.map(_tok, batched=True, remove_columns=[text_col])
            ds_val   = ds_val.map(_tok,   batched=True, remove_columns=[text_col])
            ds_test  = ds_test.map(_tok,  batched=True, remove_columns=[text_col])

            # Modelo + LoRA
            base = AutoModelForSequenceClassification.from_pretrained(base_name, num_labels=2)
            lora_cfg = LoraConfig(
                task_type=TaskType.SEQ_CLS, r=r, lora_alpha=alpha, lora_dropout=dropout, bias="none"
            )
            model = get_peft_model(base, lora_cfg)

            # TrainingArguments compatibles
            args = build_training_args(
                output_dir="./_lora_tmp",
                num_train_epochs=num_epochs,
                lr=lr,
                batch_size=batch_size,
                weight_decay=weight_decay,
                seed=RSEED,
                logging_steps=50
            )

            # Métricas para Trainer (clase 1)
            def _metrics_fn(eval_pred):
                logits, labels = eval_pred
                preds = logits.argmax(axis=-1)
                return {
                    "accuracy": accuracy_score(labels, preds),
                    "precision": precision_score(labels, preds, pos_label=1, zero_division=0),
                    "recall": recall_score(labels, preds, pos_label=1, zero_division=0),
                    "f1": f1_score(labels, preds, pos_label=1, zero_division=0),
                }

            trainer = Trainer(
                model=model,
                args=args,
                train_dataset=ds_train,
                eval_dataset=ds_val,
                tokenizer=tok,
                compute_metrics=_metrics_fn
            )

            # Entrenar
            trainer.train()

            # Threshold óptimo en validación
            logits_val = trainer.predict(ds_val).predictions
            scores_val = _sigmoid(logits_val[:, 1] if logits_val.ndim==2 else logits_val.squeeze())
            thr_opt = tune_threshold_on_val(y_val, scores_val)

            # Test
            logits_test = trainer.predict(ds_test).predictions
            scores_test = _sigmoid(logits_test[:, 1] if logits_test.ndim==2 else logits_test.squeeze())
            y_pred = (scores_test >= thr_opt).astype(int)

            # Dimensión de referencia
            n_features = int(getattr(model.base_model.config, "hidden_size", 768))
            vec_name   = f"tokenizer({os.path.basename(base_name)})"
            sampler_nm = f"OS_{oversample_ratio}" if oversample_ratio is not None else "none"
            model_nm   = f"LoRA_{os.path.basename(base_name)}_r{r}_a{alpha}_d{dropout}"

            # Fila de resultados (formato make_metrics_row)
            row = make_metrics_row(
                y_true=y_te, y_pred=y_pred, y_score=scores_test,
                dataset="test",
                model=model_nm,
                sampler=sampler_nm,
                vectorizer=vec_name,
                threshold=float(thr_opt),
                n_features=n_features,
                notes="LoRA fine-tuning (thr tuned on val)"
            )
            resultados.append(row)

            if verbose:
                msg = f"   ✓ {tag} | thr={thr_opt:.3f} | f1_1={row['f1_1'].iloc[0]:.3f} | f1_macro={row['f1_macro'].iloc[0]:.3f}"
                (tqdm.write if _HAS_TQDM else print)(msg)

        except Exception as e:
            err = f"ERROR {os.path.basename(base_name)}: {type(e).__name__}: {e}"
            if verbose:
                print("   ↪︎", err)
            resultados.append(pd.DataFrame([{
                "dataset":"test","model":f"LoRA_{os.path.basename(base_name)}_r{r}_a{alpha}_d{dropout}",
                "sampler": f"OS_{oversample_ratio}" if oversample_ratio is not None else "none",
                "vectorizer": f"tokenizer({os.path.basename(base_name)})",
                "threshold":np.nan,"n_features":np.nan,"n_samples":len(y_te),
                "prevalence_1":float((y_te==1).mean()),
                "accuracy":np.nan,"f1_micro":np.nan,"f1_macro":np.nan,"f1_weighted":np.nan,"pr_auc":np.nan,
                "precision_1":np.nan,"recall_1":np.nan,"f1_1":np.nan,
                "precision_0":np.nan,"recall_0":np.nan,"f1_0":np.nan,
                "tn":np.nan,"fp":np.nan,"fn":np.nan,"tp":np.nan,
                "notes":err
            }]))

    res = pd.concat(resultados, ignore_index=True)
    res = res.sort_values(["f1_1", "pr_auc"], ascending=False).reset_index(drop=True)
    return res

In [7]:
# ============================
# Parrilla de Experimentos LoRA
# ============================
import itertools
import pandas as pd
import numpy as np

# --- Definición de grids ---
BASE_MODELS = [
    "dccuchile/bert-base-spanish-wwm-cased",
    "xlm-roberta-base",
]

LORA_GRID = [
    {"r": 16, "alpha": 32, "dropout": 0.10},
    {"r": 32, "alpha": 64, "dropout": 0.05},
]  # puedes reactivar {"r": 8, "alpha": 16, "dropout": 0.0} si quieres

EPOCHS_GRID      = [6, 9]
LR_GRID          = [2e-5, 3e-5]
MAXLEN_GRID      = [256, 384]
OVERSAMPLE_GRID  = [0.7, 1.0]

# --- Acá guardaremos todos los resultados ---
runs = []

print(f"🧪 Ejecutando parrilla LoRA: "
      f"{len(EPOCHS_GRID)} (epochs) × {len(LR_GRID)} (lr) × {len(MAXLEN_GRID)} (maxlen) × {len(OVERSAMPLE_GRID)} (oversample) "
      f"× {len(BASE_MODELS)} (base_models) × {len(LORA_GRID)} (lora_cfgs) ≈ 64 experimentos totales.")

for num_epochs, lr, max_length, os_ratio in itertools.product(
    EPOCHS_GRID, LR_GRID, MAXLEN_GRID, OVERSAMPLE_GRID
):
    print(f"\n▶️ Corrida: epochs={num_epochs} | lr={lr} | max_len={max_length} | oversample={os_ratio}")
    res = run_lora_experiments(
        df=df,
        text_col="transcripcion",
        target_col="2.2 Alternativas de acuerdo a la necesidad",
        base_models=BASE_MODELS,
        lora_grid=LORA_GRID,
        test_size=0.2,
        val_size=0.25,
        max_length=max_length,
        num_epochs=num_epochs,
        lr=lr,
        batch_size=8,
        weight_decay=0.01,
        oversample_ratio=os_ratio,
        verbose=True
    )
    # Anotar hiperparámetros externos en cada fila
    res["epochs"]     = num_epochs
    res["lr"]         = lr
    res["max_length"] = max_length
    res["oversample"] = os_ratio
    runs.append(res)

# --- Consolidado y ranking ---
resultados_lora_ext = pd.concat(runs, ignore_index=True)

cols_principales = [
    "model","sampler","vectorizer","threshold","n_features","n_samples","prevalence_1",
    "accuracy","f1_micro","f1_macro","f1_weighted","pr_auc",
    "precision_1","recall_1","f1_1",
    "precision_0","recall_0","f1_0",
    "tn","fp","fn","tp",
    "epochs","lr","max_length","oversample","notes"
]

# Ordena por F1 de la clase positiva y luego PR-AUC
ranking = resultados_lora_ext.sort_values(["f1_1","pr_auc"], ascending=False).reset_index(drop=True)

# Vista resumida top-15
display_cols = [c for c in cols_principales if c in ranking.columns]
print("\n🏁 TOP-15 (ordenado por f1_1 y pr_auc):")
display(ranking[display_cols].head(15))

# Imprime el mejor experimento seleccionado
best = ranking.iloc[0]
print("\nMejor experimento seleccionado:")
print(f"  Modelo     : {best['model']}")
print(f"  Sampler    : {best['sampler']}")
print(f"  Vectorizer : {best['vectorizer']}")
print(f"  Threshold  : {best['threshold']:.3f}")
print(f"  F1_1       : {best['f1_1']:.3f} | F1_macro: {best['f1_macro']:.3f} | PR-AUC: {best['pr_auc']:.3f}")
print(f"  Epochs={best['epochs']} | LR={best['lr']} | max_len={int(best['max_length'])} | oversample={best['oversample']}")

🧪 Ejecutando parrilla LoRA: 2 (epochs) × 2 (lr) × 2 (maxlen) × 2 (oversample) × 2 (base_models) × 2 (lora_cfgs) ≈ 64 experimentos totales.

▶️ Corrida: epochs=6 | lr=2e-05 | max_len=256 | oversample=0.7


  0%|          | 0/4 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
50,0.704700
100,0.698400
150,0.693300
200,0.699400
250,0.697500
300,0.696900
350,0.695800
400,0.700400
450,0.695800
500,0.697600


   ✓ xlm-roberta-base|r=32|a=64|d=0.05 | thr=0.432 | f1_1=0.205 | f1_macro=0.401

🏁 TOP-15 (ordenado por f1_1 y pr_auc):


,model,sampler,vectorizer,threshold,n_features,n_samples,prevalence_1,accuracy,f1_micro,f1_macro,...,f1_0,tn,fp,fn,tp,epochs,lr,max_length,oversample,notes
0,LoRA_xlm-roberta-base_r16_a32_d0.1,OS_0.7,tokenizer(xlm-roberta-base),0.464453,768,116,0.112069,0.568966,0.568966,0.479914,...,0.695122,57,46,4,9,6,0.00003,384,0.7,LoRA fine-tuning (thr tuned on val)
1,LoRA_xlm-roberta-base_r32_a64_d0.05,OS_0.7,tokenizer(xlm-roberta-base),0.465060,768,116,0.112069,0.560345,0.560345,0.473993,...,0.687117,56,47,4,9,6,0.00003,384,0.7,LoRA fine-tuning (thr tuned on val)
2,LoRA_xlm-roberta-base_r32_a64_d0.05,OS_1.0,tokenizer(xlm-roberta-base),0.514203,768,116,0.112069,0.422414,0.422414,0.389330,...,0.531469,38,65,2,11,6,0.00003,384,1.0,LoRA fine-tuning (thr tuned on val)
3,LoRA_xlm-roberta-base_r32_a64_d0.05,OS_1.0,tokenizer(xlm-roberta-base),0.521057,768,116,0.112069,0.362069,0.362069,0.342525,...,0.455882,31,72,2,11,6,0.00002,256,1.0,LoRA fine-tuning (thr tuned on val)
4,LoRA_xlm-roberta-base_r16_a32_d0.1,OS_1.0,tokenizer(xlm-roberta-base),0.513817,768,116,0.112069,0.465517,0.465517,0.408553,...,0.592105,45,58,4,9,6,0.00003,384,1.0,LoRA fine-tuning (thr tuned on val)
5,LoRA_xlm-roberta-base_r16_a32_d0.1,OS_1.0,tokenizer(xlm-roberta-base),0.521624,768,116,0.112069,0.405172,0.405172,0.371101,...,0.517483,37,66,3,10,6,0.00003,256,1.0,LoRA fine-tuning (thr tuned on val)
6,LoRA_bert-base-spanish-wwm-cased_r32_a64_d0.05,OS_0.7,tokenizer(bert-base-spanish-wwm-cased),0.457056,768,116,0.112069,0.405172,0.405172,0.371101,...,0.517483,37,66,3,10,6,0.00002,256,0.7,LoRA fine-tuning (thr tuned on val)
7,LoRA_bert-base-spanish-wwm-cased_r16_a32_d0.1,OS_0.7,tokenizer(bert-base-spanish-wwm-cased),0.452611,768,116,0.112069,0.336207,0.336207,0.321637,...,0.421053,28,75,2,11,6,0.00003,256,0.7,LoRA fine-tuning (thr tuned on val)
8,LoRA_bert-base-spanish-wwm-cased_r32_a64_d0.05,OS_1.0,tokenizer(bert-base-spanish-wwm-cased),0.509892,768,116,0.112069,0.336207,0.336207,0.321637,...,0.421053,28,75,2,11,6,0.00002,256,1.0,LoRA fine-tuning (thr tuned on val)
9,LoRA_bert-base-spanish-wwm-cased_r16_a32_d0.1,OS_1.0,tokenizer(bert-base-spanish-wwm-cased),0.501172,768,116,0.112069,0.267241,0.267241,0.264563,...,0.308943,19,84,1,12,6,0.00002,384,1.0,LoRA fine-tuning (thr tuned on val)



Mejor experimento seleccionado:
  Modelo     : LoRA_xlm-roberta-base_r16_a32_d0.1
  Sampler    : OS_0.7
  Vectorizer : tokenizer(xlm-roberta-base)
  Threshold  : 0.464
  F1_1       : 0.265 | F1_macro: 0.480 | PR-AUC: 0.271
  Epochs=6 | LR=3e-05 | max_len=384 | oversample=0.7


In [8]:
resultados_lora_ext

,dataset,model,sampler,vectorizer,threshold,n_features,n_samples,prevalence_1,accuracy,f1_micro,...,f1_0,tn,fp,fn,tp,notes,epochs,lr,max_length,oversample
0,test,LoRA_bert-base-spanish-wwm-cased_r32_a64_d0.05,OS_0.7,tokenizer(bert-base-spanish-wwm-cased),0.457056,768,116,0.112069,0.405172,0.405172,...,0.517483,37,66,3,10,LoRA fine-tuning (thr tuned on val),6,0.00002,256,0.7
1,test,LoRA_bert-base-spanish-wwm-cased_r16_a32_d0.1,OS_0.7,tokenizer(bert-base-spanish-wwm-cased),0.428195,768,116,0.112069,0.146552,0.146552,...,0.074766,4,99,0,13,LoRA fine-tuning (thr tuned on val),6,0.00002,256,0.7
2,test,LoRA_xlm-roberta-base_r16_a32_d0.1,OS_0.7,tokenizer(xlm-roberta-base),0.451646,768,116,0.112069,0.137931,0.137931,...,0.056604,3,100,0,13,LoRA fine-tuning (thr tuned on val),6,0.00002,256,0.7
3,test,LoRA_xlm-roberta-base_r32_a64_d0.05,OS_0.7,tokenizer(xlm-roberta-base),0.450501,768,116,0.112069,0.120690,0.120690,...,0.019231,1,102,0,13,LoRA fine-tuning (thr tuned on val),6,0.00002,256,0.7
4,test,LoRA_xlm-roberta-base_r32_a64_d0.05,OS_1.0,tokenizer(xlm-roberta-base),0.521057,768,116,0.112069,0.362069,0.362069,...,0.455882,31,72,2,11,LoRA fine-tuning (thr tuned on val),6,0.00002,256,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,test,LoRA_bert-base-spanish-wwm-cased_r32_a64_d0.05,OS_0.7,tokenizer(bert-base-spanish-wwm-cased),0.424545,768,116,0.112069,0.681034,0.681034,...,0.808290,78,25,12,1,LoRA fine-tuning (thr tuned on val),9,0.00003,384,0.7
60,test,LoRA_xlm-roberta-base_r32_a64_d0.05,OS_1.0,tokenizer(xlm-roberta-base),0.432435,768,116,0.112069,0.465517,0.465517,...,0.597403,46,57,5,8,LoRA fine-tuning (thr tuned on val),9,0.00003,384,1.0
61,test,LoRA_xlm-roberta-base_r16_a32_d0.1,OS_1.0,tokenizer(xlm-roberta-base),0.432041,768,116,0.112069,0.422414,0.422414,...,0.556291,42,61,6,7,LoRA fine-tuning (thr tuned on val),9,0.00003,384,1.0
62,test,LoRA_bert-base-spanish-wwm-cased_r32_a64_d0.05,OS_1.0,tokenizer(bert-base-spanish-wwm-cased),0.511763,768,116,0.112069,0.818966,0.818966,...,0.899522,94,9,12,1,LoRA fine-tuning (thr tuned on val),9,0.00003,384,1.0


In [9]:
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
path = f"/content/drive/MyDrive/Bootcamp Llamadas Cobranza/resultados_lora_{ts}.xlsx"

# Exportar a Excel
resultados_lora_ext.to_excel(path, index=False)
# Descargar
files.download(path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
import re
import numpy as np

# Asegúrate de tener en memoria el DataFrame 'resultados_lora_ext'
assert "resultados_lora_ext" in globals(), "No encuentro 'resultados_lora_ext'. Ejecuta primero la parrilla."

# Ranking por F1 de la clase positiva y luego PR-AUC
ranking = resultados_lora_ext.sort_values(["f1_1", "pr_auc"], ascending=False).reset_index(drop=True)
best = ranking.iloc[0]

print("Mejor experimento:")
print(best[[
    "model","sampler","vectorizer","threshold","f1_1","f1_macro","pr_auc",
    "epochs","lr","max_length","oversample"
]])

# --- Parseo de hiperparámetros desde las columnas ---
# model: "LoRA_<base>_r{r}_a{alpha}_d{dropout}"
m = re.match(r"LoRA_(.+?)_r(\d+)_a(\d+)_d([0-9.]+)", best["model"])
assert m, f"No pude parsear el modelo: {best['model']}"
base_name_short, r_str, alpha_str, d_str = m.groups()

# vectorizer: "tokenizer(<base_id>)" → nos da el ID HF completo
m2 = re.match(r"tokenizer\((.+)\)", best["vectorizer"])
assert m2, f"No pude parsear el vectorizer: {best['vectorizer']}"
base_model_id = m2.group(1)

# sampler "OS_1.0" o "none"
oversample = None
if isinstance(best.get("oversample", None), (int, float)) and not np.isnan(best["oversample"]):
    oversample = float(best["oversample"])
elif isinstance(best.get("sampler", ""), str) and best["sampler"].startswith("OS_"):
    try:
        oversample = float(best["sampler"].split("_", 1)[1])
    except:
        oversample = None

BEST_CFG = {
    "base_model_id": base_model_id,
    "lora_r": int(r_str),
    "lora_alpha": int(alpha_str),
    "lora_dropout": float(d_str),
    "epochs": int(best["epochs"]) if "epochs" in best else 3,
    "lr": float(best["lr"]) if "lr" in best else 2e-5,
    "max_length": int(best["max_length"]) if "max_length" in best else 256,
    "oversample_ratio": oversample,
    "threshold": float(best["threshold"]),
}
print("\nConfig seleccionada:", BEST_CFG)

Mejor experimento:
model         LoRA_xlm-roberta-base_r16_a32_d0.1
sampler                                   OS_0.7
vectorizer           tokenizer(xlm-roberta-base)
threshold                               0.464453
f1_1                                    0.264706
f1_macro                                0.479914
pr_auc                                  0.271101
epochs                                         6
lr                                       0.00003
max_length                                   384
oversample                                   0.7
Name: 0, dtype: object

Config seleccionada: {'base_model_id': 'xlm-roberta-base', 'lora_r': 16, 'lora_alpha': 32, 'lora_dropout': 0.1, 'epochs': 6, 'lr': 3e-05, 'max_length': 384, 'oversample_ratio': 0.7, 'threshold': 0.4644532799720764}


In [11]:
import os, json, time, pandas as pd, numpy as np, warnings
warnings.filterwarnings("ignore")

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.utils import shuffle

# ------- utilidades mínimas -------
def build_training_args(output_dir, num_train_epochs, lr, batch_size=8, weight_decay=0.01, seed=42, logging_steps=50):
    # Versión mínima compatible (sin evaluation_strategy para evitar errores de compatibilidad)
    return TrainingArguments(
        output_dir=output_dir,
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=weight_decay,
        logging_steps=logging_steps,
        seed=seed,
        report_to=[]  # evita que pida credenciales de wandb
    )

def oversample_df(df_in, label_col, ratio=1.0, seed=42):
    if ratio is None:
        return df_in
    df = df_in.copy()
    y = df[label_col].astype(int).values
    n_pos, n_neg = (y==1).sum(), (y==0).sum()
    if n_pos == 0 or n_neg == 0:
        return df
    target_pos = int(round(ratio * n_neg))
    if target_pos <= n_pos:
        return df
    extra = df[df[label_col]==1].sample(n=target_pos-n_pos, replace=True, random_state=seed)
    return pd.concat([df, extra], ignore_index=True)

def tokenize_hf(df_text_label, text_col, tokenizer, max_length=256):
    ds = Dataset.from_pandas(df_text_label[["label", text_col]], preserve_index=False)
    def _tok(batch):
        return tokenizer(batch[text_col], truncation=True, padding="max_length", max_length=max_length)
    ds = ds.map(_tok, batched=True, remove_columns=[text_col])
    return ds

# ------- parámetros (desde BEST_CFG) -------
TEXT_COL   = "transcripcion"
TARGET_COL = "2.2 Alternativas de acuerdo a la necesidad"

cfg = BEST_CFG  # del bloque anterior

# ------- preparar dataset completo -------
assert TEXT_COL in df.columns and TARGET_COL in df.columns
dfi = df.dropna(subset=[TEXT_COL, TARGET_COL]).copy()
dfi[TARGET_COL] = dfi[TARGET_COL].astype(int)
dfi = shuffle(dfi, random_state=42).reset_index(drop=True)

if cfg["oversample_ratio"] is not None:
    dfi = oversample_df(dfi, TARGET_COL, ratio=cfg["oversample_ratio"], seed=42)

df_train_hf = pd.DataFrame({
    TEXT_COL: dfi[TEXT_COL].astype(str).values,
    "label":  dfi[TARGET_COL].astype(int).values
})

# ------- tokenizer / datasets -------
tok = AutoTokenizer.from_pretrained(cfg["base_model_id"], use_fast=True)
ds_train = tokenize_hf(df_train_hf, TEXT_COL, tok, max_length=cfg["max_length"])

# ------- modelo + LoRA -------
base = AutoModelForSequenceClassification.from_pretrained(cfg["base_model_id"], num_labels=2)
lconf = LoraConfig(task_type=TaskType.SEQ_CLS, r=cfg["lora_r"], lora_alpha=cfg["lora_alpha"], lora_dropout=cfg["lora_dropout"], bias="none")
model = get_peft_model(base, lconf)

# ------- train -------
out_dir = f"./lora_export_{os.path.basename(cfg['base_model_id'])}_{int(time.time())}"
args = build_training_args(out_dir, num_train_epochs=cfg["epochs"], lr=cfg["lr"], batch_size=8, weight_decay=0.01, seed=42)
trainer = Trainer(model=model, args=args, train_dataset=ds_train, tokenizer=tok)
trainer.train()

# ------- export (adapter + tokenizer + metadatos) -------
base_name = os.path.basename(cfg["base_model_id"])
timestamp = int(time.time())
export_dir = f"/content/drive/MyDrive/Bootcamp Llamadas Cobranza/lora_export_{base_name}_{timestamp}"

os.makedirs(export_dir, exist_ok=True)

# guarda solo el adapter LoRA (ligero)
model.save_pretrained(export_dir)
tok.save_pretrained(export_dir)

# metadatos (para inferencia en Streamlit)
meta = {
    "base_model_id": cfg["base_model_id"],
    "lora_r": cfg["lora_r"],
    "lora_alpha": cfg["lora_alpha"],
    "lora_dropout": cfg["lora_dropout"],
    "max_length": cfg["max_length"],
    "threshold": cfg["threshold"],  # del mejor en validación
    "label_positive": 1,
    "text_col": TEXT_COL
}
with open(os.path.join(export_dir, "metadata.json"), "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

print(f"✅ Exportado para Streamlit en: {export_dir}")

Map:   0%|          | 0/874 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
50,0.690000
100,0.680800
150,0.688200
200,0.674600
250,0.688900
300,0.677300
350,0.678600
400,0.679300
450,0.685200
500,0.678900


✅ Exportado para Streamlit en: /content/drive/MyDrive/Bootcamp Llamadas Cobranza/lora_export_xlm-roberta-base_1757384065
